<a href="https://colab.research.google.com/github/kithinji007/API-lab/blob/master/Hadover_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import pandas as pd
from tabulate import tabulate
from datetime import datetime, timedelta

In [70]:
alms = pd.read_csv('/content/June MTD ALMS.csv')

systems = pd.read_csv('/content/June MTD.csv')

In [71]:
important_cols = [0,1,2,3,10,11,13,14,16,21]
systems = systems.iloc[:, important_cols]

needed_cols = [0,1,2,3,7,8,10,11,13,17]
alms = alms.iloc[:, needed_cols]


In [72]:
display(alms.head(2)), display(systems.head(2))

,system_number,handover_at,product_name,customer_market_hub,application_form_by_contractor_number,application_form_by_contractor_name,referring_contractor_number,referring_contractor_name,referring_customer_system_numbers,system_state
0,17072018,2024-06-01 05:46:12 UTC,"MySol 10W 3 Moussa (Neo), Radio, KE",KE-Mwingi,KE26369,Florence Ndinda Musili,KE26369,Florence Ndinda Musili,NaN,sold
1,17090531,2024-06-01 07:20:56 UTC,"MySol 10W 3 (Neo, Moussa), KE",KE-18-KAPSABET,KE23587,Enock Kiprotich Kiprotich,KE23587,Enock Kiprotich Kiprotich,NaN,sold


,system_number,handover_at,product_name,customer_market_hub,application_form_by_contractor_number,application_form_by_contractor_name,referring_contractor_number,referring_contractor_name,referring_customer_system_numbers,system_state
0,SLS7L/A/BT/241400392,NaN,NaN,KE-KENDUBAY-2-STOCK POINT,KE25891,Biron Omondi Okumu,KE25891,Biron Omondi Okumu,NaN,handed_over
1,SLS7L/A/BT/241202705,NaN,NaN,KE-KAKUMA,KE24853,Masudi Igwe Juma,KE24853,Masudi Igwe Juma,NaN,handed_over


(None, None)

In [73]:
display(f"alms: {alms.shape}")

display(f'systems: {systems.shape}')

'alms: (2940, 10)'

'systems: (3263, 10)'

In [74]:
# Filter out specific product names
alms = alms[~alms['product_name'].isin(['MySol Swan Torch, KE', 'Solar Radio'])]

alms = alms.drop_duplicates(subset='system_number')


In [75]:
display(f"alms: {alms.shape}")

display(f'systems: {systems.shape}')

'alms: (2279, 10)'

'systems: (3263, 10)'

In [76]:
alms.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2279 entries, 0 to 2939
Data columns (total 10 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   system_number                          2279 non-null   object 
 1   handover_at                            2279 non-null   object 
 2   product_name                           2279 non-null   object 
 3   customer_market_hub                    2279 non-null   object 
 4   application_form_by_contractor_number  2279 non-null   object 
 5   application_form_by_contractor_name    2279 non-null   object 
 6   referring_contractor_number            2271 non-null   object 
 7   referring_contractor_name              2271 non-null   object 
 8   referring_customer_system_numbers      8 non-null      float64
 9   system_state                           2279 non-null   object 
dtypes: float64(1), object(9)
memory usage: 195.9+ KB


In [77]:
systems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 10 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   system_number                          3263 non-null   object
 1   handover_at                            3254 non-null   object
 2   product_name                           875 non-null    object
 3   customer_market_hub                    3263 non-null   object
 4   application_form_by_contractor_number  3263 non-null   object
 5   application_form_by_contractor_name    3263 non-null   object
 6   referring_contractor_number            3191 non-null   object
 7   referring_contractor_name              3191 non-null   object
 8   referring_customer_system_numbers      70 non-null     object
 9   system_state                           3263 non-null   object
dtypes: object(10)
memory usage: 255.0+ KB


In [78]:
# drop column with system_no not an int
systems = systems[systems['system_number'].str.match(r'^\d+$')]

In [79]:
# change system system_number to alms' dtype
# systems.loc[:, 'system_number'] = systems['system_number'].astype(alms['system_number'].dtype)

In [80]:
# alms.info()

In [81]:
# systems.info()

In [82]:
# Join systems and alm on system_id using merge
sales = pd.merge(systems, alms[['system_number', 'product_name']], on='system_number', how='left')


# Update product_name in systems with the one from ALMS if it is null
sales['product_name'] = sales['product_name_x'].combine_first(sales['product_name_y'])

# Drop the extra columns from merging
sales.drop(columns=['product_name_x', 'product_name_y'], inplace=True)

sales.sample(5)

,system_number,handover_at,customer_market_hub,application_form_by_contractor_number,application_form_by_contractor_name,referring_contractor_number,referring_contractor_name,referring_customer_system_numbers,system_state,product_name
1276,17170697,2024-06-11 12:00:04 UTC,KE-KITUI,KE25433,Patrick Kabwere,KE25209,Selestine Mango,NaN,sold,"Mysol 55W TV 24"", KE"
2055,17213513,2024-06-15 16:17:19 UTC,KE-KINANGO-STOCK POINT,KE23075,Kengo Nyae Tsuma,KE23075,Kengo Nyae Tsuma,NaN,sold,"MySol 10W 3 Moussa (Neo), Radio, KE"
3196,172780512,2024-06-23 17:48:30 UTC,KE-KINANGO-STOCK POINT,KE22133,Nehemiah Mwadalu Meri,KE22133,Nehemiah Mwadalu Meri,NaN,sold,"MySol 10W 3 Moussa (Neo), Radio, KE"
2593,17249335,2024-06-19 18:44:34 UTC,KE-KOMBANI,KE21893,Fatuma Rashid Gakweli,KE21893,Fatuma Rashid Gakweli,NaN,sold,"MySol 10W 3 Moussa (Neo), Radio, KE"
1627,17189333,2024-06-13 09:59:15 UTC,KE-KINANGO-STOCK POINT,KE22631,Mrinzi Mrabu Daniel,KE22631,Mrinzi Mrabu Daniel,NaN,sold,"MySol 10W 3 Moussa (Neo), Radio, KE"


In [83]:
sales['product_name'].value_counts()

product_name
MySol 10W 3 Moussa (Neo), Radio, KE                                 2022
MySol 80W TV 32, KE                                                  290
Mysol 55W TV 24", KE                                                 249
MySol 10W 3 (Neo), KE                                                108
Refurbished MySol 10W 3 Neo (Moussa), KE                              99
Mysol 55W Basic, KE                                                   89
MySol 120W TV 42 (R T), KE                                            55
MySol 80W 3, KE                                                       52
MySol 10W 3 (Neo, Moussa), KE                                         49
Refurbished MySol 120W TV 42 - (RT), KE                               44
MySol 200W 6, KE                                                      23
MySol 10W 2 (Neo), KE                                                 12
Refurb 80W + 120W TV 42 RT                                            11
MySol 200W TV 42 (R T), KE            

In [84]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3197 entries, 0 to 3196
Data columns (total 10 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   system_number                          3197 non-null   object
 1   handover_at                            3197 non-null   object
 2   customer_market_hub                    3197 non-null   object
 3   application_form_by_contractor_number  3197 non-null   object
 4   application_form_by_contractor_name    3197 non-null   object
 5   referring_contractor_number            3126 non-null   object
 6   referring_contractor_name              3126 non-null   object
 7   referring_customer_system_numbers      69 non-null     object
 8   system_state                           3197 non-null   object
 9   product_name                           3134 non-null   object
dtypes: object(10)
memory usage: 249.9+ KB


In [85]:
def clean_data(sales):
    # Drop rows with null product_name
    sales = sales.dropna(subset=['product_name'])

    # Only pick rows in which system_state is 'sold'
    sales = sales[sales['system_state'] == 'sold']

    # convert datetime to date
    sales['handover_at'] = pd.to_datetime(sales['handover_at'], utc=True).dt.strftime('%Y-%m-%d')

    # If referer is null assign applicant
    sales['referring_contractor_number'] = sales['referring_contractor_number'].fillna(sales['application_form_by_contractor_number'])
    sales['referring_contractor_name'] = sales['referring_contractor_name'].fillna(sales['application_form_by_contractor_name'])

    # Replace CRC with 'First Sale' if null, otherwise with 'CRC'
    sales['referring_customer_system_numbers'] = sales['referring_customer_system_numbers'].fillna('First Sale')
    sales['referring_customer_system_numbers'] = sales['referring_customer_system_numbers'].apply(lambda x: 'CRC' if x != 'First Sale' else x)

    # Renaming columns
    sales = sales.rename(columns={
                                  'handover_at':'handover',
                                  'product_name':'product',
                                  'customer_market_hub':'hub',
                                  'application_form_by_contractor_number':'applicant_id',
                                  'application_form_by_contractor_name':'applicant_name',
                                  'referring_contractor_number':'referrer_id',
                                  'referring_contractor_name':'referrer_name',
                                  'referring_customer_system_numbers':'sale_type'
                                  })

    sales['product'] = sales['product'].str.extract(r'(\d+W)')

    return sales

sales = clean_data(sales)
sales.head()

,system_number,handover,hub,applicant_id,applicant_name,referrer_id,referrer_name,sale_type,system_state,product
0,17072018,2024-06-01,KE-Mwingi,KE26369,Florence Ndinda Musili,KE26369,Florence Ndinda Musili,First Sale,sold,10W
1,17090275,2024-06-01,KE-07-HOMABAY,KE24501,Christine Atieno Anindo,KE24501,Christine Atieno Anindo,First Sale,sold,55W
2,17090531,2024-06-01,KE-18-KAPSABET,KE23587,Enock Kiprotich Kiprotich,KE23587,Enock Kiprotich Kiprotich,First Sale,sold,10W
3,170907117,2024-06-01,KE-ROMBO,KE25741,James Muinde Mwendwa,KE25741,James Muinde Mwendwa,First Sale,sold,10W
4,170876114,2024-06-01,KE-MALINDI,KE24663,Pethwel Maitha,KE24663,Pethwel Maitha,First Sale,sold,10W


In [86]:
# Define the mapping dictionary
mapping = {
"KE-03-KITALE"	:	"North Rift 1",
"KE-04-KISUMU"	:	"Western 1"	,
"KE-01-KAKAMEGA " : "Western 1",
"KE-05-BUSIA"	:	"Western 2"	,
"(Missing stock)KITALE"	:	"North Rift 1"	,
"KE-06-BUNGOMA"	:	"North Rift 1"	,
"KE-07-HOMABAY"	:	"Nyanza"	,
"KE-09-SIAYA"	:	"Western 2"	,
"KE-10-KAJIADO"	:	"Kajiado"	,
"KE-13-MAUA"	:	"Eastern 1"	,
"KE-17-MIGORI"	:	"Nyanza"	,
"KE-18-KAPSABET"	:	"North Rift 2"	,
"KE-77-Ochamgei Wholesales-3PD" :	"North Rift 2"	,
"KE-19-MBITA"	:	"Nyanza"	,
"KE-20-BONDO"	:	"Western 2"	,
"KE-21-LUANDA"	:	"Western 1"	,
"KE-25-OYUGIS"	:	"Nyanza"	,
"KE-26-MOI'S BRIDGE"	:	"North Rift 2"	,
"KE-Awendo"	:	"Nyanza"	,
"KE-Cheptais-2-Stock Point"	:	"North Rift 1"	,
"KE-ENDEBES"	:	"North Rift 1"	,
"KE-Funyula-2-Stock Point"	:	"Western 2"	,
"KE-KAKUMA"	:	"Kakuma"	,
"KE-KALOLENI-STOCK POINT"	:	"Kilifi/Coast 2"	,
"KE-Kathwana-Stock Point"	:	"Eastern 2"	,
"KE-KIBWEZI"	:	"Makueni"	,
"KE-KENDUBAY-2-STOCK POINT"	:	"Nyanza"	,
"KE-KIMANA-STOCK POINT"	:	"Kajiado"	,
"KE-KIMILILI-STOCK POINT"	:	"Western 2"	,
"KE-Kiminini-Stock Point"	:	"North Rift 1"	,
"KE-KINANGO-STOCK POINT"	:	"Kwale/Coast 1"	,
"KE-KITALE"	:	"North Rift 1"	,
"KE-KITUI"	:	"Kitui 1"	,
"KE-KOMBANI"	:	"Kwale/Coast 1"	,
"KE-KYUSO"	:	"Eastern 2"	,
"KE-Lolgorian-Stock Point"	:	"South-Rift"	,
"KE-LUNGALUNGA STOCK POINT"	:	"Kwale/Coast 1"	,
"KE-Makutano Kapenguria Stock Point"	:	"North Rift 1"	,
"KE-MALINDI"	:	"Kilifi/Coast 2"	,
"KE-Maraba-Stock Point"	:	"North Rift 2"	,
"KE-Marimanti-Stock Point"	:	"Eastern 2"	,
"KE-MATANO MANNE  STOCK POINT"	:	"Kilifi/Coast 2"	,
"KE-12-MERU"	:	"Eastern 1"	,
"KE-MERU TOWN"	:	"Eastern 1"	,
"KE-Muhoroni-Stock Point"	:	"Western 1"	,
"KE-MUTITU"	:	"Kitui 2"	,
"KE-MUTOMO STOCK POINT"	:	"Kitui 1"	,
"KE-Mwingi"	:	"Kitui 2"	,
"KE-NAIROBI-STORE"	:	"Telesales"	,
"KE-Ndhiwa-Stock Point"	:	"Nyanza"	,
"KE-Ndiwa-2-Stock Point"	:	"Nyanza"	,
"KE-Nyandhiwa/Magunga-Stock Point"	:	"Nyanza"	,
"KE-Olokurto-Stock Point"	:	"South-Rift"	,
"KE-Ololung'a-Stock point"	:	"South-Rift"	,
"KE-Orolwa Stock Point"	:	"North Rift 1"	,
"KE-Oyugis-2-Stock Point"	:	"Nyanza"	,
"KE-PORT VICTORIA"	:	"Western 2"	,
"KE-SABATIA-STOCK POINT"	:	"Western 1"	,
"KE-SIAKAGO"	:	"Eastern 1"	,
"KE-Suswa"	:	"South-Rift"	,
"KE-UGUNJA"	:	"Western 2"	,
"KE-KIRITIRI"	:	"Eastern 1"	,
"KE-KAWANGARE"	:	"Makueni"	,
"KE-ROMBO"	:	"Kajiado"	,
"KE-BAMBA" : "Kajiado",
"KE-KADONGO STOCKPOINT" : "Nyanza",
"KE-RAGENGNI CLUSTER STORE" : "Western 2",
"KE-KABUOCH STOCKPOINT" : "Nyanza",
"KE-MALABA CLUSTER STORE" : "Western 2",
"KE-VIBANDANI KWA BITA -CLUSTER STORE" : "Kwale/Coast 1",
"KE-KADONGO STOCKPOINT" : "Nyanza",
"KE-SAMBURU -CLUSTER STORE" : "Kwale/Coast 1",
"KE-TSEIKURU CLUSTER STORE" : "Eastern 2",
"KE-MIGWANI STOCKPOINT" : "Kitui 2",
"KE-KILIFI STOCKPOINT" : "Kilifi/Coast 2",
"KE-MATSANGONI CLUSTER STORE" : "Kilifi/Coast 2",
"KE-SEGA CLUSTER STORE" : "Western 2",
"KE-MAKINA STOCKPOINT" : "Kwale/Coast 1",
"KE-CINEMA CLUSTER STORE" :  "Nyanza",
"KE-SHIMBA HILLS-CLUSTER STORE" : "Kwale/Coast 1",
"KE-MARAFA CLUSTER STORE" :  "Kilifi/Coast 2",
"KE-KIKIMA CLUSTER STORE" : "Makueni",
"KE-TINGA STOCKPOINT" : "Kajiado",
"KE-USENGE CLUSTER STORE" : "Western 2",
"KE-KOMBEWA CLUSTER STORE" : "Western 1",
"KE-MUMIAS CLUSTER STORE" : "Western 1",
"KE-KATITO STOCKPOINT" : "Western 1",
"KE-YALA CLUSTER STORE" : "Western 2",
"KE-SIMBA CEMENT CLUSTER STORE" : "Kajiado"
}

In [87]:
sales['Region'] = sales['hub'].map(mapping)

In [88]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3134 entries, 0 to 3196
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   system_number   3134 non-null   object
 1   handover        3134 non-null   object
 2   hub             3134 non-null   object
 3   applicant_id    3134 non-null   object
 4   applicant_name  3134 non-null   object
 5   referrer_id     3134 non-null   object
 6   referrer_name   3134 non-null   object
 7   sale_type       3134 non-null   object
 8   system_state    3134 non-null   object
 9   product         3134 non-null   object
 10  Region          3134 non-null   object
dtypes: object(11)
memory usage: 358.4+ KB


In [89]:
# # Get rows where the 'Region' column has null values
# null_regions = sales[sales['Region'].isnull()]

# # Display the rows with null values in the 'Region' column
# null_regions.sample()


In [90]:
# sales.to_csv('Cleaned_data.csv', index=False)

In [91]:
sales_per_region = sales.groupby(['Region','product']).size().reset_index(name='sales_count')
sales_per_region = sales_per_region.sort_values(by='sales_count', ascending=False)


# Pivot the DataFrame
pivot_table = sales_per_region.pivot_table(index='Region', columns='product', values='sales_count', aggfunc='sum').fillna(0)

# Use tabulate to print the pivot table with lines
print(tabulate(pivot_table, headers='keys', tablefmt='psql'))

+----------------+-------+--------+--------+-------+-------+-------+-------+
| Region         |   10W |   120W |   200W |   40W |   50W |   55W |   80W |
|----------------+-------+--------+--------+-------+-------+-------+-------|
| Eastern 1      |    84 |      1 |      1 |     0 |     1 |    18 |    13 |
| Eastern 2      |   148 |      0 |      1 |     0 |     0 |    15 |    11 |
| Kajiado        |   236 |      4 |      2 |     0 |     0 |    24 |    31 |
| Kakuma         |   108 |      0 |      0 |     0 |     0 |     0 |     2 |
| Kilifi/Coast 2 |   138 |      3 |      2 |     0 |     1 |    11 |    12 |
| Kitui 1        |   120 |      0 |      1 |     0 |     0 |    13 |     3 |
| Kitui 2        |   160 |      0 |      0 |     0 |     0 |     7 |     5 |
| Kwale/Coast 1  |   604 |     14 |      3 |     0 |     1 |    51 |    35 |
| Makueni        |    97 |      0 |      1 |     0 |     0 |     8 |     3 |
| North Rift 1   |    49 |      3 |      3 |     0 |     1 |    37 |    15 |

In [92]:
sales_per_region2 = sales.groupby('Region').size().reset_index(name='sales_count')
sales_per_region2 = sales_per_region2.sort_values(by='sales_count', ascending=False)


print(tabulate(sales_per_region2, headers='keys', tablefmt='psql', showindex=False))

+----------------+---------------+
| Region         |   sales_count |
|----------------+---------------|
| Kwale/Coast 1  |           708 |
| Western 1      |           355 |
| Nyanza         |           335 |
| Kajiado        |           297 |
| Western 2      |           212 |
| Eastern 2      |           175 |
| Kitui 2        |           172 |
| Kilifi/Coast 2 |           167 |
| Kitui 1        |           137 |
| Eastern 1      |           118 |
| Kakuma         |           110 |
| Makueni        |           109 |
| North Rift 1   |           108 |
| North Rift 2   |            89 |
| Telesales      |            42 |
+----------------+---------------+


In [93]:
agents = [
    "KE25863",
    "KE25935",
    "KE25739",
    "KE25831",
    "KE25525",
    "KE26489",
    "KE26009",
    "KE26493",
    "KE25879",
    "KE25133",
    "KE26487",
    "KE23521",
    "KE26491",
    "KE26551",
    "KE26527",
    "KE26521",
    "KE26833",
    "KE26877",
    "KE26879",
    "KE26831",
    "KE25969",
    "KE26883",
    "KE26851",
    "KE26865",
    "ke26869",
    "KE26873",
    "KE26887",
    "KE26889",
    "KE26891",
    "KE26893",
    "KE26895",
    "KE26897",
    "KE26899",
    "KE26901",
    "KE26903",
    "KE24709",
    "KE26517",
    "KE26541",
    "KE26525",
    "KE26575",
    "KE26533",
    "KE26531",
    "KE26529",
    "KE26535",
    "KE26565",
    "KE26513",
    "KE26515",
    "KE26537",
    "KE26609",
    "KE26207",
    "KE24935",
    "KE26961",
    "KE22949"

]

In [94]:
# parasol = sales[sales['referrer_id'].isin(agents)]

In [95]:
parasol2 = sales[sales['applicant_id'].isin(agents) | sales['referrer_id'].isin(agents)]

In [96]:
# Group by 'Region' and 'Product' and count the sales
parasol2 = parasol2.groupby(['handover', 'applicant_id', 'applicant_name', 'referrer_id', 'referrer_name' ,'Region', 'product']).size().reset_index(name='sales')

print(tabulate(parasol2, headers='keys', tablefmt='psql', showindex=False))

+------------+----------------+----------------------------+---------------+----------------------------+---------------+-----------+---------+
| handover   | applicant_id   | applicant_name             | referrer_id   | referrer_name              | Region        | product   |   sales |
|------------+----------------+----------------------------+---------------+----------------------------+---------------+-----------+---------|
| 2024-06-01 | KE25739        | Malaki Wesechere Sheunda   | KE22513       | Patrick Shitandi Amukowa   | Western 1     | 10W       |       1 |
| 2024-06-01 | KE26487        | Gladys Awuor Ogalo         | KE26573       | Benard Oyoo Osunga         | Western 1     | 10W       |       1 |
| 2024-06-01 | KE26491        | Favour Maina               | KE19907       | Paul Masaba Ombewa         | Western 1     | 10W       |       1 |
| 2024-06-02 | KE25831        | Hannah Wambui              | KE25917       | Calvin Mark                | Western 1     | 10W       |   

In [66]:
# Convert 'handover' column to datetime format
parasol2['handover'] = pd.to_datetime(parasol2['handover'], errors='coerce')  # 'coerce' will handle any errors, like non-date strings

# Calculate yesterday's date
yesterday = datetime.now() - timedelta(days=1)
yesterday_date = yesterday.date()

# Filter sales for yesterday
sales_yesterday = parasol2[pd.to_datetime(parasol2['handover']).dt.date == yesterday_date]

# Display the filtered DataFrame
sales_yesterday.head(9)


,system_number,handover,applicant_id,applicant_name,referrer_id,referrer_name,Region,product,sales
96,17273731,2024-06-22,KE26207,Allan Omondi Ooko,KE26207,Allan Omondi Ooko,Nyanza,10W,1
97,17275397,2024-06-22,KE26533,Mercy Atieno,KE26011,Edward Ochieng Otieno,Nyanza,10W,1
99,17276650,2024-06-22,KE26513,Kennedy Ooko Ogola,KE24695,Collete Achieng Okute,Nyanza,10W,1
100,17276759,2024-06-22,KE26883,Metrine Matsatsa,KE25131,Spencer Wawire,Western 1,10W,1
101,17276890,2024-06-22,KE25739,Malaki Wesechere Sheunda,KE22513,Patrick Shitandi Amukowa,Western 1,10W,1


In [67]:
parasol2 = parasol2.groupby('Region').size().reset_index(name='sales')
parasol2 = parasol2.sort_values(by='sales', ascending=False)


print(tabulate(parasol2, headers='keys', tablefmt='psql', showindex=False))

+---------------+---------+
| Region        |   sales |
|---------------+---------|
| Western 1     |      65 |
| Nyanza        |      28 |
| Kwale/Coast 1 |      12 |
+---------------+---------+


In [494]:
parasol_products = parasol2.groupby(['Region', 'product']).size().reset_index(name='sales')

# Pivot the DataFrame
pivot_table = parasol_products.pivot_table(index='Region', columns='product', values='sales', aggfunc='sum').fillna(0)

# Use tabulate to print the pivot table with lines
print(tabulate(pivot_table, headers='keys', tablefmt='psql'))

+---------------+-------+--------+-------+-------+
| Region        |   10W |   120W |   55W |   80W |
|---------------+-------+--------+-------+-------|
| Kwale/Coast 1 |    11 |      0 |     0 |     1 |
| Nyanza        |    13 |      3 |     3 |     3 |
| Western 1     |    42 |      0 |     6 |     3 |
+---------------+-------+--------+-------+-------+


In [78]:
# parasol.to_csv('parasol_data.csv', index=False)